<a href="https://colab.research.google.com/github/Wayne-wyyking888/Stat-8931-GenAI/blob/main/chapter2/Chapter_II_Large_language_Modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Packages, Paths & Environment

### (1). Download the dependent .py files; trained model; tokenizer to the **currect directory**

In [ ]:
# download model.py and tokenizer.py
! gdown --id 1SU7jSZI36KGwBv5-zgc3WkStPK6lGKwL -O /content/model.py # download model.py
! gdown --id 1uXCgdmip79J6efM5hiHGCy9mdr_U8BXT -O /content/tokenizer.py # download tokenizer.py

/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1SU7jSZI36KGwBv5-zgc3WkStPK6lGKwL
From (redirected): https://drive.google.com/uc?id=1SU7jSZI36KGwBv5-zgc3WkStPK6lGKwL&confirm=t&uuid=e1de7aff-babd-4ab8-a628-d104d4cab4da
To: /content/model.py
100% 13.3k/13.3k [00:00<00:00, 36.5MB/s]
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1uXCgdmip79J6efM5hiHGCy9mdr_U8BXT
From (redirected): https://drive.google.com/uc?id=1uXCgdmip79J6efM5hiHGCy9mdr_U8BXT&confirm=t&uuid=a4e0a780-ba5b-44c9-95b1-16475e49bef0
To: /content/tokenizer.py
100

In [2]:
# trained language model (checkpoints) and tokenizer files under suitable directories
# (1) make directories
! mkdir /content/models
! mkdir /content/data
# (2) download checkpoint and tokenizers
! gdown --id 1npTlkOP_TtW-oEMROTDrYXzOjLL2yuUQ -O /content/models/trained_model.pt
! gdown --id 1LezVTflV_Irp0GQn0Rzgn3F77tppozGa -O /content/data/trained_tokenizer.model



mkdir: cannot create directory ‘/content/models’: File exists
mkdir: cannot create directory ‘/content/data’: File exists
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1npTlkOP_TtW-oEMROTDrYXzOjLL2yuUQ
From (redirected): https://drive.google.com/uc?id=1npTlkOP_TtW-oEMROTDrYXzOjLL2yuUQ&confirm=t&uuid=9e157553-b01b-409c-bb04-8c95d308aa2b
To: /content/models/trained_model.pth
100% 104M/104M [00:00<00:00, 136MB/s] 
/usr/local/lib/python3.10/dist-packages/gdown/__main__.py:132: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1LezVTflV_Irp0GQn0Rzgn3F77tppozGa
To: /content/data/trained_tokenizer.mode

### (2) Load pretrained model and test
* Load pretrained model
* Adjust TF32 precision
* Config parameters for generation & decoding

In [8]:
from contextlib import nullcontext
import torch
from model import ModelArgs, Transformer
from tokenizer import Tokenizer
import os

checkpoint = os.getcwd() + "/trained_model.pt"
tokenizer = os.getcwd() + "/data/trained_tokenizer.model"

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Use device: {device}")

# load checkpoint
checkpoint_dict = torch.load(checkpoint, map_location=device)
gptconf = ModelArgs(**checkpoint_dict['model_args'])
model = Transformer(gptconf)
state_dict = checkpoint_dict['model']
unwanted_prefix = '_orig_mod.' #the unwanted prefix was sometimes added during compiling
for k,v in list(state_dict.items()):
    if k.startswith(unwanted_prefix):
        state_dict[k[len(unwanted_prefix):]] = state_dict.pop(k)
model.load_state_dict(state_dict, strict=False)
model.eval()
model.to(device)

# load tokenizer
enc = Tokenizer(tokenizer_model=tokenizer)

# adjust precision
dtype = 'bfloat16' if torch.cuda.is_available() and torch.cuda.is_bf16_supported() else 'float16' # 'float32' or 'bfloat16' or 'float16'
torch.backends.cuda.matmul.allow_tf32 = True #enables the use of TF32 for matrix multiplication operations within PyTorch when using CUDA
torch.backends.cudnn.allow_tf32 = True #enables the use of TF32 precision within the cuDNN library
device_type = 'cuda' if 'cuda' in device else 'cpu'
ptdtype = {'float32': torch.float32, 'bfloat16': torch.bfloat16, 'float16': torch.float16}[dtype]
ctx = nullcontext() if device_type == 'cpu' else torch.amp.autocast(device_type=device_type, dtype=ptdtype)

Use device: cpu


<ipython-input-8-42fd5ddb1fe9>:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint_dict = torch.load(checkpoint, map_location=device)


RuntimeError: PytorchStreamReader failed reading zip archive: failed finding central directory

In [10]:
! stat /content/trained_model.pt

  File: /content/trained_model.pt
  Size: 88080384  	Blocks: 172040     IO Block: 4096   regular file
Device: 2fh/47d	Inode: 4980878     Links: 1
Access: (0644/-rw-r--r--)  Uid: (    0/    root)   Gid: (    0/    root)
Access: 2024-09-11 21:38:15.371371658 +0000
Modify: 2024-09-11 21:36:51.963609463 +0000
Change: 2024-09-11 21:36:52.005612868 +0000
 Birth: 2024-09-11 21:36:28.899739586 +0000


In [6]:
stat $checkpoint

SyntaxError: invalid syntax (<ipython-input-6-18a79ac15844>, line 1)

In [ ]:
## parameter configurations
num_samples = 1 # number of samples to draw
max_new_tokens = 256 # number of tokens generated in each sample
temperature = 1.0 # 1.0 = no change, < 1.0 = less random, > 1.0 = more random, in predictions
top_k = 300 # retain only the top_k most likely tokens



In [ ]:
# Generate texts
start = "Once upon a time, there is a unicorn"
start_ids = enc.encode(start, bos=True, eos=False)
x = (torch.tensor(start_ids, dtype=torch.long, device=device)[None, ...])
with torch.no_grad():
    with ctx:
        for k in range(num_samples):
            y = model.generate(x, max_new_tokens, temperature=temperature, top_k=top_k)
            print(enc.decode(y[0].tolist()))
            print('---------------')

In [ ]:
## ** check if the same devices

print("Device of model parameters:", next(model.parameters()).device)
print("Device of input tensor 'x':", x.device)

.  ..  .config	.ipynb_checkpoints  sample_data  Stat-8931-GenAI


total 4
drwxr-xr-x 1 root root 4096 Sep 10 13:28 sample_data


In [ ]:
with open('https://drive.google.com/file/d/1SU7jSZI36KGwBv5-zgc3WkStPK6lGKwL/view', 'r') as _:
  _

FileNotFoundError: [Errno 2] No such file or directory: 'https://drive.google.com/file/d/1SU7jSZI36KGwBv5-zgc3WkStPK6lGKwL/view'